Before you turn this lab in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

**Provide your name and any collaborators below:**

In [ ]:
// YOUR CODE HERE
???

# Lab 4 - Functional Programming
> Labs will be due each week before the homework. They are not intended take a significant amount of time but rather to provide examples/practice on specific and isolated features in the language. Labs are autograded so you can get quick feedback.

### Import the necessary Chisel dependencies. 
> There will be a cell like this in every lab. Make sure you run it before proceeding to bring the Chisel Library into the Jupyter Notebook scope!

In [ ]:
interp.load.module(os.Path(s"${System.getProperty("user.dir")}/resource/chisel_deps.sc"))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

## Problem 1 (3 pts) - Seq addition
> Use Scala `zip` and `map` to add the contents of two `Seq`s (element by element).

In [ ]:
def addSeqs(a: Seq[Int], b: Seq[Int]): Seq[Int] = {
    // YOUR CODE HERE
    ???
}

In [ ]:
val a = Seq.tabulate(8)(_.toInt)
val b = Seq.tabulate(8)(_.toInt)
assert(addSeqs(a, b) == Seq(0, 2, 4, 6, 8, 10, 12, 14))


## Problem 2 (3 pts) - foreach with Chisel
> The `VecRotate` module below shifts its input `Vec` by a constant `offset` (wraps around). Complete it by using `foreach`. You may find the Scala methods `drop` and `take` helpful.

In [ ]:
class VecRotate(numElems: Int, width: Int, offset: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Vec(numElems, UInt(width.W)))
        val out = Output(Vec(numElems, UInt(width.W)))
    })
    val rotated = io.in.drop(offset) ++ io.in.take(offset)
    // YOUR CODE HERE
    ???
}

In [ ]:
def testVecRotate(numElems: Int, width: Int): Boolean = {
    for (offset <- 0 until numElems) {
        val input = 0 until numElems
        val expected = input.drop(offset) ++ input.take(offset)
        test(new VecRotate(numElems, width, offset)) { dut =>
            (0 until numElems).foreach{ i => dut.io.in(i).poke(input(i).U) }
            (0 until numElems).foreach{ i => dut.io.out(i).expect(expected(i).U) }
        }
    }
    true
}

assert(testVecRotate(4,8))

## Problem 3 (6 pts) - zipWithIndex
> First, use `foldLeft` to implement the `exp` function (computes exponent). Then use `zipWithIndex`, `map`, `exp`, and `reduce`/`foldLeft` to concisely evaluate a polynomial. The index in the sequence is the degree in the polynomial (e.g. _coefs(i) * x^i_)

In [ ]:
def exp(base: Int, deg: Int): Int = {
    // YOUR CODE HERE
    ???
}

def polyEval(coefs: Seq[Int], x: Int): Int = {
    // YOUR CODE HERE
    ???
}

In [ ]:
assert (exp(5, 0) == 1)
assert (exp(2, 5) == 32)
assert (exp(4, 3) == 64)
// 0*x^0 + 1*x^1 + 2*x^2
assert(polyEval(Seq(0, 1, 2), 5) == 55)
assert(polyEval(Seq(0, 1, 2), 0) == 0)


## Problem 4 (6 pts) - map on matrix
> Given a `n` x `n` matrix (`Seq[Seq[Int]]`), use `map` and `zipWithIndex` to add `x` to the diagonal (other cells unchanged). For this function `incDiag`, the matrix is in row-major order. For example if `x=4`: 
``` 
    List(1, 1, 1, 1, 1) -> List(5, 1, 1, 1, 1)
    List(1, 1, 1, 1, 1) -> List(1, 5, 1, 1, 1)
    List(1, 1, 1, 1, 1) -> List(1, 1, 5, 1, 1)
    List(1, 1, 1, 1, 1) -> List(1, 1, 1, 5, 1)
    List(1, 1, 1, 1, 1) -> List(1, 1, 1, 1, 5)
```

In [ ]:
// YOUR CODE HERE
???

In [ ]:
val in = List(
  List(1, 1, 1, 1, 1),
  List(1, 1, 1, 1, 1),
  List(1, 1, 1, 1, 1),
  List(1, 1, 1, 1, 1),
  List(1, 1, 1, 1, 1)
)

val out = List(
  List(5, 1, 1, 1, 1),
  List(1, 5, 1, 1, 1),
  List(1, 1, 5, 1, 1),
  List(1, 1, 1, 5, 1),
  List(1, 1, 1, 1, 5)
)
assert(incDiag(in, 4) == out)



## Problem 5 (10 pts) - flatMap and reduce with Chisel
> Let's put together what we've covered to make a Chisel module. Complete the `MatrixSearch` module below that looks for the input `searchFor` by comparing all of the elements of the input matrix `mat` (2D `Vec`). If (and only if) `searchFor` matches any of the elements in `mat`, the output `found` should be _true_. Your solution should use `flatMap`, `reduce`, and possibly `map`.

In [ ]:
class MatrixSearch(numRows: Int, numCols: Int, width: Int) extends Module {
    require(numRows > 1)
    require(numCols > 1)
    val io = IO(new Bundle {
        val mat = Input(Vec(numRows, Vec(numCols, UInt(width.W))))
        val searchFor = Input(UInt(width.W))
        val found = Output(Bool())
    })
    // YOUR CODE HERE
    ???
}

In [ ]:
def testMatrixSearch(numRows: Int, numCols: Int, width: Int): Boolean = {
    require(log2Ceil(numRows) < width)
    test(new MatrixSearch(numRows, numCols, width)) { dut =>
        (0 until numRows) foreach {
             r => (0 until numCols) foreach { 
                c => dut.io.mat(r)(c).poke(r.U)
            }
        }
        for (r <- 0 until numRows) {
            dut.io.searchFor.poke(r.U)
            dut.io.found.expect(true.B)
        }
        dut.io.searchFor.poke(numRows.U)
        dut.io.found.expect(false.B)
    }
    true
}

assert(testMatrixSearch(2,2,8))
